In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
​
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
​
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
​
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
​
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

SyntaxError: invalid character in identifier (<ipython-input-43-4f1e9015b3a3>, line 4)

In [ ]:
# Импортируем все необходимые модули
import warnings
import random
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import roc_auc_score, r2_score as r2, SCORERS
from sklearn.preprocessing import StandardScaler, RobustScaler
#from xgboost import XGBRegressor
#from lightgbm import LGBMRegressor
import seaborn as sns
#from sklearn.manifold import TSNE

In [ ]:
%matplotlib inline
warnings.filterwarnings('ignore')
matplotlib.rcParams.update({'font.size': 14})
pd.pandas.set_option('display.max_columns', None)
sns.set_style('darkgrid')

In [ ]:
# Для оценки каждой модели создадим функцию вывода метрики R2 и визуализации расчетной и фактической стоимости
def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values):
    print('Train R2:\t' + str(round(r2(train_true_values, train_pred_values), 3)))
    print('Test R2:\t' + str(round(r2(test_true_values, test_pred_values), 3)))
    
    plt.figure(figsize = (18, 10))
    
    plt.subplot(121)
    sns.scatterplot(x = train_pred_values, y = test_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Train sample prediction')
    
    plt.subplot(122)
    sns.scatterplot(x = test_pred_values, y = test_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Test sample prediction')
    
    plt.show()

In [ ]:
# Для оптимизации скорости работы с данными создадим функцию уменьшения объема памяти, который занимает датасет
def reduce_mem_usage(df):
    # определим тип данных в каждой колонке датасета, для типов семейства int и float изменим разрядность в соответствии с минимальным и максимальным значением колонки        
    
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
 
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
# создадим функцию для изучения и визуализации вещественных признаков 
def learn_real_features(df, feature_name):
    if df[feature_name].dtype.name in ['float64', 'float32', 'float16', 'int8', 'int16', 'int32']:
        plt.figure(figsize = (16, 8))
        sns.distplot(df[feature_name])
        y = np.linspace(0, 0.000005, 10)
        feature_mean = round(df[feature_name].mean(), 2)
        feature_median = df[feature_name].median()
        feature_mode = df[feature_name].mode()[0]
        feature_min = round(df[feature_name].min(), 2)
        feature_max = round(df[feature_name].max(), 2)
        feature_NA_number = df[feature_name].isnull().sum()
        plt.plot([feature_mean] * 10, y, label='mean',  linewidth=4)
        plt.plot([feature_median] * 10, y, label='median',  linewidth=4)
        plt.plot([feature_mode] * 10, y, label='mode', linewidth=4)
        plt.title('Distribution of {} '.format(feature_name))
        plt.legend()
        print(f'median - {feature_median}\nmean - {feature_mean}\nmode - {feature_mode}\nMin - {feature_min}\nMax - {feature_max}\nNA number - {feature_NA_number}')
        plt.show()
    else:
        print("Признак не является вещественным")

In [ ]:
def change_type(df, feature_type):
    for col in feature_type.keys():
        df[col] = df[col].astype(feature_type[col])
    return df

In [ ]:
def binary_to_numbers(df, binary_dict):
    feature_list = binary_dict.keys()
    for col in feature_list:
        df[col] = df[col].map(binary_dict[col])
    return df

In [ ]:
# создадим класс для простой (базовой) предобработки датасета
class SimplePreprocessing:
    """Подготовка исходных данных"""

    def __init__(self):
        """Аттрибуты класса"""
        self.medians = None
        self.tops = None
        self.numeric_cols = None
        self.non_numeric_cols = None
        self.drop_col_list = None
        self.kitchen_square_quantile = None
        self.kitchen_square_min = None
        self.tech_square_min = None
                
    def fit(self, X, drop_list=[]):
        """Сохранение статистик"""       
        # Расчет медиан
        self.medians = X.median() # медианные значения признаков датасета
        self.numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist() # список признаков и числовыми типами значений
        self.non_numeric_cols = X.select_dtypes(exclude=[np.number]).columns.tolist() # список признаков и нечисловыми типами значений
        self.tops = X[self.non_numeric_cols].describe().loc['top'] # самое часто встречаемое значение признака
        self.drop_col_list = drop_list # список признаков, неиспользуемых в обучении модели
        self.kitchen_square_quantile = X['KitchenSquare'].quantile(.975)
        self.kitchen_square_min = 2
        self.tech_square_min = 3
    
    def transform(self, X):
        """Трансформация данных"""
        # Rooms       
        X.loc[X['Rooms'] == 0, 'Rooms'] = 1
        X.loc[X['Rooms'] >= 10, 'Rooms'] = self.medians['Rooms']
        
        # KitchenSquare
        condition_out = (X['KitchenSquare'] > self.kitchen_square_quantile)
        X.loc[condition_out, 'KitchenSquare'] = self.medians['KitchenSquare']
        X.loc[X['KitchenSquare'] < self.kitchen_square_min, 'KitchenSquare'] = self.kitchen_square_min
        condition_na = (X['KitchenSquare'].isna()) & \
                      (~X['Square'].isna()) & \
                      (~X['LifeSquare'].isna())
        X.loc[condition_na, 'KitchenSquare'] = X.loc[condition_na, 'Square'] - X.loc[condition_na, 'LifeSquare'] - self.tech_square_min
        
        # HouseFloor
        X.loc[X['HouseFloor'] == 0, 'HouseFloor'] = self.medians['HouseFloor']
        
        # Floor
        floor_outliers = X.loc[X['Floor'] > X['HouseFloor']].index
        X.loc[floor_outliers, 'Floor'] = X.loc[floor_outliers, 'HouseFloor'].apply(lambda x: random.randint(1, x))
        
        # HouseYear
        current_year = datetime.now().year
        X.loc[X['HouseYear'] > current_year, 'HouseYear'] = current_year
        
        # LifeSquare
        condition = (X['LifeSquare'].isna()) & \
                      (~X['Square'].isna()) & \
                      (~X['KitchenSquare'].isna())
        X.loc[condition, 'LifeSquare'] = X.loc[condition, 'Square'] - X.loc[condition, 'KitchenSquare'] - self.tech_square_min
        
        # универсальная обработка пропусков по всем признакам
        for col in self.numeric_cols:
            missing = X[col].isnull()
            pers_missing = np.mean(X[col].isnull())*100
            if pers_missing >= 30:
                X.drop(columns=[col], inplace=True)
                print('из датасета удален признак: {}, т.к. процент пропусков {} выше 30%'.format(col, pers_missing))
            elif pers_missing > 0 and pers_missing < 30:  
                X['{}_ismissing'.format(col)] = 0
                X.loc[X[col] == None, '{}_ismissing'.format(col)] = 1
                med = self.medians[col]
                X[col] = X[col].fillna(med)
                print('заполняем пропуски для признака: {} значением {}'.format(col, med))
        
        for col in self.non_numeric_cols:
            missing = X[col].isnull()
            pers_missing = np.mean(X[col].isnull())*100
            if pers_missing >= 30:
                X.drop(columns=[col], inplace=True)
                print('из датасета удален признак: {}, т.к. процент пропусков {} выше 30%'.format(col, pers_missing))
            elif pers_missing > 0 and pers_missing < 30:
                X['{}_ismissing'.format(col)] = 0
                X.loc[X[col] == None, '{}_ismissing'.format(col)] = 1
                top = self.tops[col]
                df[col] = X[col].fillna(top)
                print('заполняем пропуски для признака: {} значением {}'.format(col, top))
        
        # удаление неиспользуемых в обучении признаков
        for col in self.drop_col_list:
            X.drop(columns=[col], inplace=True)
            print('Из датасета удален признак: {} '.format(col))
        
        return X

In [ ]:
# пути к директориям файлов с данными
TRAIN_DATASET_PATH = '../input/real-estate-price-prediction-moscow/train.csv'
TEST_DATASET_PATH = '../input/real-estate-price-prediction-moscow/test.csv'
SUBMISSION_DATASET_PATH = '../input/real-estate-price-prediction-moscow/sample_submission.csv'
TRAIN_DATASET_PREPARED_PATH = './train_prepared.csv'
TEST_DATASET_PREPARED_PATH = './test_prepared.csv'
TRAIN_DATASET_BASE_PREPARED_PATH = './train_base_prepared.csv'
TEST_DATASET_BASE_PREPARED_PATH = './test_base_prepared.csv'

In [ ]:
# загрузим исходные данные в датасеты pandas и применим к ним функцию reduce_mem_usage
train_df = pd.read_csv(TRAIN_DATASET_PATH)
test_df = pd.read_csv(TEST_DATASET_PATH)
submit_df = pd.read_csv(SUBMISSION_DATASET_PATH)
target_name = 'Price'

FileNotFoundError: [Errno 2] No such file or directory: '../input/real-estate-price-prediction-moscow/train.csv'

In [ ]:
# https://www.kaggle.com/vladislavargun/house-price-prediction/edit